In [1]:
import numpy as np

In [2]:
Reply = np.array([
    [1, 5, 5, 0, 3],
    [2, 0, 5, 0, 3],
    [0, 0, 0, 0, 0],
    [2, 0, 0, 4, 0],
    [0, 4, 2, 0, 0]
])

Reply

array([[1, 5, 5, 0, 3],
       [2, 0, 5, 0, 3],
       [0, 0, 0, 0, 0],
       [2, 0, 0, 4, 0],
       [0, 4, 2, 0, 0]])

In [3]:
Retweet = np.array([
    [0, 2, 4, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 4, 3],
    [0, 2, 5, 0, 0]
])

Retweet

array([[0, 2, 4, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 4, 3],
       [0, 2, 5, 0, 0]])

In [4]:
Like = np.array([
    [2, 5, 1, 0, 3],
    [0, 0, 0, 0, 0],
    [0, 5, 0, 3, 3],
    [1, 0, 0, 4, 1],
    [0, 0, 9, 0, 0]
])

Like

array([[2, 5, 1, 0, 3],
       [0, 0, 0, 0, 0],
       [0, 5, 0, 3, 3],
       [1, 0, 0, 4, 1],
       [0, 0, 9, 0, 0]])

In [5]:
Post = np.array([2, 5, 10, 4, 3])

Post

array([ 2,  5, 10,  4,  3])

# Engagement Statistics

In [6]:
# 1. Find the user who replies more than others.
reply_out = Reply.sum(axis=1)

reply_out.argmax()

0

In [7]:
# 2. Find the user who retweets more than others.
retweet_out = Retweet.sum(axis=1)

retweet_out == retweet_out.max()

array([False, False, False,  True,  True])

In [8]:
# 3. Find users who neither reply nor retweet.
reply_retweet_out = reply_out + retweet_out

reply_retweet_out == 0

array([False, False,  True, False, False])

In [9]:
# 4. Display the number of likes each user has given to their own tweets.

# Create a diagonal matrix 
I = np.eye(Like.shape[0])

nbr_self_likes = (I * Like).sum(axis=1)

nbr_self_likes

array([2., 0., 0., 4., 0.])

In [10]:
# 5. Display the number of likes each user has given to tweets other than their own.

like_out = Like.sum(axis=1)

nbr_notself_likes = like_out - nbr_self_likes

nbr_notself_likes

array([ 9.,  0., 11.,  2.,  9.])

In [11]:
# 6. Find users who like their own tweets more than they like others' tweets.

like_more_self = nbr_notself_likes <  nbr_self_likes

like_more_self

array([False, False, False,  True, False])

In [12]:
# 7. Calculate their number.

like_more_self.astype(int).sum()

1

In [13]:
# 8. Display the number of interactions each user has made.

reply_retweet_out + like_out

array([31, 10, 11, 19, 22])

# Interaction Statistics

In [14]:
# 1. Display the maximum number of likes each user has received (from users other than themselves).

max_likes_in = Like.max(axis=0)

max_likes_in

array([2, 5, 9, 4, 3])

In [15]:
# 2. Find users who have received likes for all their tweets by at least one person other than themselves.

max_likes_in == Post

array([ True,  True, False,  True,  True])

In [16]:
# 3. Calculate the number of incoming edges (having at least one interaction).

in_arcs = ((Reply + Retweet + Like) > 0).astype(int).sum(axis=0)

in_arcs

array([3, 3, 3, 2, 4])

In [17]:
# 4. Calculate the number of outgoing edges (having at least one interaction).
out_arcs = ((Reply + Retweet + Like) > 0).astype(int).sum(axis=1)

out_arcs

array([4, 3, 3, 3, 2])

In [18]:
# 5. Find nodes where the number of incoming edges is equal to the number of outgoing edges.

in_more_out = in_arcs == out_arcs

in_more_out

array([False,  True,  True, False, False])

In [19]:
# 6. Display the likes they have given to each other.

Like[:, in_more_out][in_more_out, :]

array([[0, 0],
       [5, 0]])

# Popularity

In [20]:
I1 = 1 - I

I1

array([[0., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1.],
       [1., 1., 0., 1., 1.],
       [1., 1., 1., 0., 1.],
       [1., 1., 1., 1., 0.]])

In [21]:
# We are not interested in reflexive interactions
# unless for retweets, we are only interested in reflexive interactions

Reply_nodiag = Reply * I1 # no diagonal
Retweet_diag = Retweet * I # only diagonal
Like_nodiag = Like * I1 # no diagonal

Reply_nodiag, Retweet_diag, Like_nodiag

(array([[0., 5., 5., 0., 3.],
        [2., 0., 5., 0., 3.],
        [0., 0., 0., 0., 0.],
        [2., 0., 0., 0., 0.],
        [0., 4., 2., 0., 0.]]),
 array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 4., 0.],
        [0., 0., 0., 0., 0.]]),
 array([[0., 5., 1., 0., 3.],
        [0., 0., 0., 0., 0.],
        [0., 5., 0., 3., 3.],
        [1., 0., 0., 0., 1.],
        [0., 0., 9., 0., 0.]]))

In [22]:
# For each interaction, 
# if the number of Replies received from a person is more than their Likes, we multiply the number of Replies by (-0.5). 
# If it is equal, we multiply their own Likes by (-0.25). 
# Otherwise, it has no effect (0).

Reply_score = np.where(Reply_nodiag > Like_nodiag, Reply_nodiag * (-0.5), np.where(Reply_nodiag == Like_nodiag, 0, Reply_nodiag) )

Reply_score

array([[ 0. ,  0. , -2.5,  0. ,  0. ],
       [-1. ,  0. , -2.5,  0. , -1.5],
       [ 0. ,  0. ,  0. ,  0. ,  0. ],
       [-1. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. , -2. ,  2. ,  0. ,  0. ]])

In [23]:
# if someone retweets their own tweets, they will be penalized by (-0.25) for each retweet.

Retweet_score = Retweet_diag * (-0.25)

Retweet_score

array([[-0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0.],
       [-0., -0., -0., -1., -0.],
       [-0., -0., -0., -0., -0.]])

In [24]:
# For each interaction, 
# if the number of Likes received from a person is greater than half of the tweets published, we multiply it by (1.5). 
# Otherwise, the score is the number of Likes.

Post_half = Post/2
Post_repeated = np.ones(Like.shape)
Post_repeated = Post_repeated * Post_half


Like_score = np.where(Like_nodiag > Post_repeated, Like_nodiag * 1.5, Like_nodiag)
Like_score

array([[ 0. ,  7.5,  1. ,  0. ,  4.5],
       [ 0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  7.5,  0. ,  4.5,  4.5],
       [ 1. ,  0. ,  0. ,  0. ,  1. ],
       [ 0. ,  0. , 13.5,  0. ,  0. ]])

In [25]:
# Calculate the popularity score for each user for each interaction which is the average of the received scores.

detaild_popularity_score = (Reply_score + Retweet_score + Like_score)/3

detaild_popularity_score

array([[ 0.        ,  2.5       , -0.5       ,  0.        ,  1.5       ],
       [-0.33333333,  0.        , -0.83333333,  0.        , -0.5       ],
       [ 0.        ,  2.5       ,  0.        ,  1.5       ,  1.5       ],
       [ 0.        ,  0.        ,  0.        , -0.33333333,  0.33333333],
       [ 0.        , -0.66666667,  5.16666667,  0.        ,  0.        ]])

In [26]:
# Calculate the final score which is the average of the scores from different interactions.

popularity_score = detaild_popularity_score.mean(axis=0)

popularity_score

array([-0.06666667,  0.86666667,  0.76666667,  0.23333333,  0.56666667])